# Set Up

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import tpot
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RepeatedStratifiedKFold
from pandas import read_csv
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import tensorflow as tf
from pandas.plotting import scatter_matrix
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcPar

In [2]:
msft = yf.Ticker("aapl")

In [3]:
df = msft.history(period = 'max')

In [4]:
df = df.reset_index()

# Feature Engineering

In [5]:
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1980-12-12,0.100326,0.100762,0.100326,0.100326,469033600,0.00,0.0
1,1980-12-15,0.095528,0.095528,0.095092,0.095092,175884800,0.00,0.0
2,1980-12-16,0.088548,0.088548,0.088112,0.088112,105728000,0.00,0.0
3,1980-12-17,0.090293,0.090729,0.090293,0.090293,86441600,0.00,0.0
4,1980-12-18,0.092911,0.093347,0.092911,0.092911,73449600,0.00,0.0
...,...,...,...,...,...,...,...,...
10371,2022-01-31,169.943485,174.777323,169.294303,174.557602,115541600,0.00,0.0
10372,2022-02-01,173.788575,174.617520,172.090741,174.387817,86213900,0.00,0.0
10373,2022-02-02,174.527647,175.656214,173.109456,175.616257,84914300,0.00,0.0
10374,2022-02-03,174.257984,176.015754,171.900986,172.679993,89418100,0.00,0.0


In [6]:
df_decade = df[df['Date'] > '2010-01-01'].reset_index().drop(columns = 'index')

In [7]:
df_decade['Close_pct_change'] = df_decade['Close'].pct_change()

In [8]:
df_decade

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Close_pct_change
0,2010-01-04,6.526949,6.559671,6.494839,6.544686,493729600,0.00,0.0,NaN
1,2010-01-05,6.562730,6.593005,6.521445,6.556002,601904800,0.00,0.0,0.001729
2,2010-01-06,6.556004,6.581998,6.444994,6.451722,552160000,0.00,0.0,-0.015906
3,2010-01-07,6.475575,6.483220,6.393005,6.439794,477131200,0.00,0.0,-0.001849
4,2010-01-08,6.431231,6.483219,6.393311,6.482607,447610800,0.00,0.0,0.006648
...,...,...,...,...,...,...,...,...,...
3040,2022-01-31,169.943485,174.777323,169.294303,174.557602,115541600,0.00,0.0,0.026126
3041,2022-02-01,173.788575,174.617520,172.090741,174.387817,86213900,0.00,0.0,-0.000973
3042,2022-02-02,174.527647,175.656214,173.109456,175.616257,84914300,0.00,0.0,0.007044
3043,2022-02-03,174.257984,176.015754,171.900986,172.679993,89418100,0.00,0.0,-0.016720


In [9]:
df_decade["Price_weekly_increase_future"] = df_decade['Close'].pct_change(periods = -5)

In [10]:
df_decade["Close_daily_increase_past"] = df_decade['Close'].pct_change(periods = 1)
df_decade["Volume_daily_increase_past"] = df_decade['Volume'].pct_change(periods = 1)
df_decade["Open_daily_increase_past"] = df_decade['Open'].pct_change(periods = 1)
df_decade["High_daily_increase_past"] = df_decade['High'].pct_change(periods = 1)
df_decade["Low_daily_increase_past"] = df_decade['Low'].pct_change(periods = 1)

df_decade["Close_weekly_increase_past"] = df_decade['Close'].pct_change(periods = 5)
df_decade["Volume_weekly_increase_past"] = df_decade['Volume'].pct_change(periods = 5)
df_decade["Open_weekly_increase_past"] = df_decade['Open'].pct_change(periods = 5)
df_decade["High_weekly_increase_past"] = df_decade['High'].pct_change(periods = 5)
df_decade["Low_weekly_increase_past"] = df_decade['Low'].pct_change(periods = 5)

In [11]:
df_decade['High_Low_range'] = df_decade["High"] - df_decade["Low"]

In [12]:
def start_week(day):
    start = day - timedelta(days=day.weekday())
    if day == start:
        return 1
    else:
        return 0
def end_week(day):
    start = day - timedelta(days=day.weekday())
    end = start + timedelta(days=4)
    if day == end:
        return 1
    else:
        return 0

In [13]:
df_decade['is_start_week'] = df_decade['Date'].apply(start_week)
df_decade['is_end_week'] = df_decade['Date'].apply(end_week)

In [14]:
df_decade['is_dividends'] = np.where(df_decade['Dividends'] > 0, 1, 0)
df_decade['is_stock_splits'] = np.where(df_decade['Stock Splits'] > 0, 1, 0)

In [15]:
df_decade['Price_weekly_increase_future'] = df_decade['Price_weekly_increase_future'].apply(lambda x: -1*x)

In [16]:
df_decade['Month'] = df_decade["Date"].apply(lambda x: x.month)

In [17]:
df_dummie_month = pd.get_dummies(df_decade['Month'])

In [18]:
df_dummie_month.columns = [f"Month_{col}" for col in df_dummie_month]

In [19]:
df_decade = df_decade.join(df_dummie_month)

In [20]:
df_decade.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'Close_pct_change', 'Price_weekly_increase_future',
       'Close_daily_increase_past', 'Volume_daily_increase_past',
       'Open_daily_increase_past', 'High_daily_increase_past',
       'Low_daily_increase_past', 'Close_weekly_increase_past',
       'Volume_weekly_increase_past', 'Open_weekly_increase_past',
       'High_weekly_increase_past', 'Low_weekly_increase_past',
       'High_Low_range', 'is_start_week', 'is_end_week', 'is_dividends',
       'is_stock_splits', 'Month', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
       'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
       'Month_11', 'Month_12'],
      dtype='object')

In [21]:
df_decade['Price_weekly_increase_future'].head()

0   -0.018562
1   -0.032062
2   -0.001519
3   -0.005491
4   -0.029379
Name: Price_weekly_increase_future, dtype: float64

In [22]:
df_decade = df_decade.dropna()

In [23]:
# create a list of our conditions
conditions = [
    (df_decade['Price_weekly_increase_future'] >=0) & (df_decade['Price_weekly_increase_future'] < 0.001),
    (df_decade['Price_weekly_increase_future'] >=0.001) & (df_decade['Price_weekly_increase_future'] < 0.01),
    (df_decade['Price_weekly_increase_future'] >=0.01 ) & ( df_decade['Price_weekly_increase_future'] < 0.02),
    (df_decade['Price_weekly_increase_future'] >=0.02 ) & ( df_decade['Price_weekly_increase_future'] < 0.03),
    (df_decade['Price_weekly_increase_future'] >=0.03),
    (df_decade['Price_weekly_increase_future'] <=0 ) & ( df_decade['Price_weekly_increase_future'] > -0.001),
    (df_decade['Price_weekly_increase_future'] <=-0.001 ) & ( df_decade['Price_weekly_increase_future'] > -0.01),
    (df_decade['Price_weekly_increase_future'] <=-0.01 ) & ( df_decade['Price_weekly_increase_future'] > -0.02),
    (df_decade['Price_weekly_increase_future'] <=-0.02 ) & ( df_decade['Price_weekly_increase_future'] > -0.03),
    (df_decade['Price_weekly_increase_future'] <=-0.03),
    
    ]

# create a list of the values we want to assign for each condition
values = ['insignificant increase < 0.1%', 'less than 1% increase', '1% - 2%', '2% - 3%', '3%+',
         'insignificant decrease > -0.1%', 'less than 1% decrease', 'decrease 1%-2%', 'decrease 2%-3%', 'decrease more that 3%']

# create a new column and use np.select to assign values to it using our lists as arguments
df_decade['tier'] = np.select(conditions, values)

# Machine Learning

#### Model Classification

In [24]:
features = ['Close_pct_change', 'Volume_daily_increase_past',
       'Open_daily_increase_past', 'High_daily_increase_past',
       'Low_daily_increase_past', 'Close_weekly_increase_past',
       'Volume_weekly_increase_past', 'Open_weekly_increase_past',
       'High_weekly_increase_past', 'Low_weekly_increase_past',
       'High_Low_range', 'is_start_week', 'is_end_week', 'is_dividends',
       'is_stock_splits', 'Month', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
       'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
       'Month_11', 'Month_12']
target = ['tier']

#### Model One-V-All

In [25]:
target_one_v = ['did_increase']

In [26]:
df_decade['Price_weekly_increase_future'] >= 0

5        True
6        True
7       False
8       False
9       False
        ...  
3035     True
3036     True
3037     True
3038     True
3039     True
Name: Price_weekly_increase_future, Length: 3035, dtype: bool

In [27]:
df_decade['did_increase'] = np.where(df_decade['Price_weekly_increase_future'] >= 0, 1,0)

In [28]:
df_decade['did_increase'].value_counts()

1    1785
0    1250
Name: did_increase, dtype: int64

### Modelling

In [29]:
def build_NN(target, model_name, epochs):
    X = df_decade[features].values
    y = df_decade[target].values
    _array =  np.unique(y, return_counts = True)[0]
    y_indices = [list(_array).index(i) for i in y]
    X_train, X_validation, Y_train, Y_validation = train_test_split(
        X,
        y_indices,
        test_size=0.20,
        random_state=1,
        shuffle=True
    )
    
    X_validation_numpy = np.asarray(X_validation)
    X_train_numpy = np.asarray(X_train)
    Y_train_numpy = np.asarray(Y_train).astype('int32')
    Y_validation_numpy = np.asarray(Y_validation).astype('int32')

    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(655, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(len(_array), activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(X_train_numpy, Y_train_numpy, epochs=epochs)
    model.evaluate(X_validation_numpy, Y_validation_numpy, verbose=2)
    model.summary()

    model.save(model_name)
    print("Success!")

In [30]:
build_NN(target_one_v, 'if_increase', 10)

Epoch 1/10
76/76 [==============================] - 1s 2ms/step - loss: 0.6853 - accuracy: 0.5898
Epoch 2/10
76/76 [==============================] - 0s 2ms/step - loss: 0.6757 - accuracy: 0.5906
Epoch 3/10
76/76 [==============================] - 0s 2ms/step - loss: 0.6744 - accuracy: 0.5906
Epoch 4/10
76/76 [==============================] - 0s 2ms/step - loss: 0.6710 - accuracy: 0.5972
Epoch 5/10
76/76 [==============================] - 0s 2ms/step - loss: 0.6620 - accuracy: 0.6058
Epoch 6/10
76/76 [==============================] - 0s 2ms/step - loss: 0.6587 - accuracy: 0.6137
Epoch 7/10
76/76 [==============================] - 0s 2ms/step - loss: 0.6576 - accuracy: 0.6079
Epoch 8/10
76/76 [==============================] - 0s 2ms/step - loss: 0.6564 - accuracy: 0.6075
Epoch 9/10
76/76 [==============================] - 0s 2ms/step - loss: 0.6513 - accuracy: 0.6190
Epoch 10/10
76/76 [==============================] - 0s 2ms/step - loss: 0.6510 - accuracy: 0.6219
19/19 - 1s - loss: 

In [31]:
build_NN(target, 'class_prediction', 100)

Epoch 1/100
76/76 [==============================] - 0s 2ms/step - loss: 2.1517 - accuracy: 0.2002
Epoch 2/100
76/76 [==============================] - 0s 2ms/step - loss: 2.0984 - accuracy: 0.2348
Epoch 3/100
76/76 [==============================] - 0s 2ms/step - loss: 2.0889 - accuracy: 0.2290
Epoch 4/100
76/76 [==============================] - 0s 2ms/step - loss: 2.0719 - accuracy: 0.2335
Epoch 5/100
76/76 [==============================] - 0s 2ms/step - loss: 2.0612 - accuracy: 0.2327
Epoch 6/100
76/76 [==============================] - 0s 2ms/step - loss: 2.0531 - accuracy: 0.2414
Epoch 7/100
76/76 [==============================] - 0s 2ms/step - loss: 2.0437 - accuracy: 0.2418
Epoch 8/100
76/76 [==============================] - 0s 2ms/step - loss: 2.0284 - accuracy: 0.2438
Epoch 9/100
76/76 [==============================] - 0s 2ms/step - loss: 2.0253 - accuracy: 0.2488
Epoch 10/100
76/76 [==============================] - 0s 2ms/step - loss: 2.0144 - accuracy: 0.2500
Epoch 11/

76/76 [==============================] - 0s 2ms/step - loss: 1.6861 - accuracy: 0.3736
Epoch 84/100
76/76 [==============================] - 0s 2ms/step - loss: 1.6904 - accuracy: 0.3814
Epoch 85/100
76/76 [==============================] - 0s 2ms/step - loss: 1.6902 - accuracy: 0.3727
Epoch 86/100
76/76 [==============================] - 0s 2ms/step - loss: 1.6838 - accuracy: 0.3764
Epoch 87/100
76/76 [==============================] - 0s 2ms/step - loss: 1.6754 - accuracy: 0.3826
Epoch 88/100
76/76 [==============================] - 0s 2ms/step - loss: 1.6664 - accuracy: 0.3843
Epoch 89/100
76/76 [==============================] - 0s 2ms/step - loss: 1.6682 - accuracy: 0.3797
Epoch 90/100
76/76 [==============================] - 0s 2ms/step - loss: 1.6593 - accuracy: 0.3818
Epoch 91/100
76/76 [==============================] - 0s 2ms/step - loss: 1.6650 - accuracy: 0.3871
Epoch 92/100
76/76 [==============================] - 0s 2ms/step - loss: 1.6565 - accuracy: 0.3764
Epoch 93/100


## Preparing For Integration

In [32]:
def format_df(df_decade, features):
    df_decade["Price_weekly_increase_future"] = df_decade['Close'].pct_change(periods = -5)
    df_decade["Close_pct_change"] = df_decade['Close'].pct_change(periods = 1)
    df_decade["Volume_daily_increase_past"] = df_decade['Volume'].pct_change(periods = 1)
    df_decade["Open_daily_increase_past"] = df_decade['Open'].pct_change(periods = 1)
    df_decade["High_daily_increase_past"] = df_decade['High'].pct_change(periods = 1)
    df_decade["Low_daily_increase_past"] = df_decade['Low'].pct_change(periods = 1)

    df_decade["Close_weekly_increase_past"] = df_decade['Close'].pct_change(periods = 5)
    df_decade["Volume_weekly_increase_past"] = df_decade['Volume'].pct_change(periods = 5)
    df_decade["Open_weekly_increase_past"] = df_decade['Open'].pct_change(periods = 5)
    df_decade["High_weekly_increase_past"] = df_decade['High'].pct_change(periods = 5)
    df_decade["Low_weekly_increase_past"] = df_decade['Low'].pct_change(periods = 5)

    df_decade['High_Low_range'] = df_decade["High"] - df_decade["Low"]
    df_decade['is_start_week'] = df_decade['Date'].apply(start_week)
    df_decade['is_end_week'] = df_decade['Date'].apply(end_week)
    
    df_decade['is_dividends'] = np.where(df_decade['Dividends'] > 0, 1, 0)
    df_decade['is_stock_splits'] = np.where(df_decade['Stock Splits'] > 0, 1, 0)
    df_decade['Price_weekly_increase_future'] = df_decade['Price_weekly_increase_future'].apply(lambda x: -1*x)
    df_decade['Month'] = df_decade["Date"].apply(lambda x: x.month)
    df_dummie_month = pd.get_dummies(df_decade['Month'])
    df_dummie_month.columns = [f"Month_{col}" for col in df_dummie_month]
    df_decade = df_decade.join(df_dummie_month)
    return df_decade[features]

In [33]:
def predicit_to_series(model, X, _array):
    predictions = model(X)
    top_k_values, top_k_indices = tf.nn.top_k(predictions, k=1)
    string_predictions = [_array[i] for i in top_k_indices]
    probabilities = [float(i) for i in top_k_values]
    return string_predictions, probabilities

In [34]:
def predict_latest_possible(model, df, _array, features):
    latest_date = format_df(df, features = features)[-1:]
    _class, probability = predicit_to_series(model, latest_date.values, _array = _array)
    return _class, probability

In [35]:
new_model = tf.keras.models.load_model('class_prediction')

new_model_if_increase = tf.keras.models.load_model('if_increase')

_array = np.array(['1% - 2%', '2% - 3%', '3%+', 'decrease 1%-2%', 'decrease 2%-3%',
       'decrease more that 3%', 'insignificant decrease > -0.1%',
       'insignificant increase < 0.1%', 'less than 1% decrease',
       'less than 1% increase'])

_array_if_increase = np.array([0,1])


features = ['Close_pct_change', 'Volume_daily_increase_past',
       'Open_daily_increase_past', 'High_daily_increase_past',
       'Low_daily_increase_past', 'Close_weekly_increase_past',
       'Volume_weekly_increase_past', 'Open_weekly_increase_past',
       'High_weekly_increase_past', 'Low_weekly_increase_past',
       'High_Low_range', 'is_start_week', 'is_end_week', 'is_dividends',
       'is_stock_splits', 'Month', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
       'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
       'Month_11', 'Month_12']

In [36]:
predict_latest_possible(model = new_model_if_increase, df = df, _array = _array_if_increase, features = features)

<ipython-input-33-76441995433c>:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  string_predictions = [_array[i] for i in top_k_indices]


([0], [0.7198010683059692])

# Strategy Testing

In [37]:
features_df = df_decade[features]

In [38]:
predictions, probabilities = predicit_to_series(new_model, features_df.values, _array)

<ipython-input-33-76441995433c>:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  string_predictions = [_array[i] for i in top_k_indices]


In [39]:
predictions_one, probabilities_one = predicit_to_series(new_model_if_increase, features_df.values, _array_if_increase)

<ipython-input-33-76441995433c>:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  string_predictions = [_array[i] for i in top_k_indices]


In [40]:
df_decade['class_prediction'] = predictions
df_decade['binary_prediction'] = predictions_one

In [41]:
df_decade['class_prediction'].value_counts()

3%+                               940
1% - 2%                           725
decrease more that 3%             396
decrease 1%-2%                    235
less than 1% increase             233
less than 1% decrease             197
2% - 3%                           191
decrease 2%-3%                     90
insignificant decrease > -0.1%     20
insignificant increase < 0.1%       8
Name: class_prediction, dtype: int64

In [42]:
test_df = df_decade[df_decade['binary_prediction'] == 1]

In [43]:
test_df = test_df[test_df['class_prediction'] == '3%+']

In [44]:
test_df['did_increase'].value_counts()

1    665
0    270
Name: did_increase, dtype: int64

In [45]:
test_df['did_increase'].value_counts()[1]/(test_df['did_increase'].value_counts()[0] + test_df['did_increase'].value_counts()[1])

0.7112299465240641